In [1199]:
import numpy as np
import pandas as pd
import re
from fuzzywuzzy import fuzz

In [1103]:
df = pd.read_excel(r"C:\Users\mfgas\Downloads\Salaries in Egypt at 2025 (الردود).xlsx")

In [1104]:
df.dtypes

Date-Time                datetime64[ns]
Title                            object
Years of Experiences             object
Salary                           object
Date of Salary                   object
What Is your Company             object
Work Type                        object
Work Hour                        object
City of Company site             object
dtype: object

In [1107]:
df.isna().sum()

Date-Time                 0
Title                     0
Years of Experiences      0
Salary                    0
Date of Salary            0
What Is your Company      0
Work Type                 0
Work Hour                 0
City of Company site     54
dtype: int64

In [1108]:
df = df.sort_values(by=["Title"])

In [1111]:
df['What Is your Company'].unique()

array(['Not Egyption but site in egypt', '.',
       'Not Egyption and site out of egypt', 'Egyption',
       'Fully remote operations', 'Ag',
       'Saudi backdoor (dev team in Egypt)', 'Else', 'corporate',
       'Startup in canada , but there’s no site just online work and the whole team from Egypt ',
       'Multinational ', 'DXC', 'Egyptian but all work outside Egypt ',
       'morocco', "I don't own a company ",
       'Egyptian and site out of Egypt '], dtype=object)

In [1113]:
df["Title"] = df["Title"].astype(str).str.strip().str.replace(r"\s+", " ", regex=True)
df["Title"] = df["Title"].astype(str).str.lstrip()

In [1115]:
df = df.drop(index=450)

In [1117]:
df["Title"] = df["Title"].astype(str).str.strip().str.replace(r"\s+", " ", regex=True).str.title()
df = df.sort_values(by="Title").reset_index(drop=True)

In [1119]:
df["Title"] = df["Title"].astype(str).str.replace(r"\bsenior\b", "", case=False, regex=True).str.strip()

In [1121]:
df = df.drop(columns='Date-Time')
df

,Title,Years of Experiences,Salary,Date of Salary,What Is your Company,Work Type,Work Hour,City of Company site
0,. Net Developer,6,1200 usd,2025-07-16 00:00:00,Not Egyption but site in egypt,Remotley,Full Time,Cairo
1,. Net Developer,2,400 USD,2025-07-07 00:00:00,Not Egyption and site out of egypt,Remotley,Full Time,Riyadh
2,. Net Developer,6 months,13000,2025-07-18 00:00:00,Egyption,On Site,Full Time,Cairo
3,.Net,8month,10,2025-07-17 00:00:00,Egyption,Remotley,Full Time,Tanta
4,.Net Backend Developer,7 months,15000,2025-06-01 00:00:00,Egyption,On Site,Full Time,Giza
...,...,...,...,...,...,...,...,...
767,Warehouse Manager,7,25k net,2025-01-02 00:00:00,Egyption,On Site,Full Time,Giza
768,Web Developer,0,8500,2025-01-01 00:00:00,Egyption,Remotley,Full Time,NaN
769,Web Scraping Specialist,5,1500 $,2023-03-15 00:00:00,Not Egyption and site out of egypt,Hybird,Full Time,Kuwait
770,Wordpress Developer,2,10000,2024-10-01 00:00:00,Egyption,Remotley,Full Time,Cairo


In [1123]:
df['Salary'].unique()

array(['1200 usd', '400 USD', 13000, 10, 15000, 50000, '17k', 14000,
       '20k', 12000, 40000, 20000, 25000, 10000, '30k', '35k', 22500,
       60000, 32000, 8000, '25k', '40 k', '14000 EGP', '1000 usd', 11.5,
       7000, 7500, 30, 30000, 17000, 57000, 38000, 1150, 4000,
       '1000$ gross', 65, 15, '25 K', '1602 USD', '40K', 80000, 6000,
       9000, 14500, 22000, 75000, 58000, 24200, '600 $', 28000, '1200$',
       42000, 3000, 23000, 34000, 36000, 18000, '200$', '25000 EGP',
       13500, 45000, '2200 USD', '40,000 EGP', 31000, '50k', 27000, 51678,
       24400, 40, '1k$', 27750, 24000, 5000, '47100 Net', 1, 15800, 65000,
       'Net - 14000 EGP', 10500, 28500,
       '15500 Egp , (in Riyadh) 7500 SAR', '3800EUR', '1900 usd',
       '7500 AED ', '34k', 16000, 27500, 8500, '18k EGP', '7k', 22, 7800,
       13, 35000, 6500, 55000, '100000 EGP', 37000, 130000,
       '55000 egp net', '660 USD', ' Net:35K', 12500, 100000, 81000,
       48000, 95000, 70000, 39000, '28000 Eyp', 16500,

In [1137]:
conversion_rates_to_egp = {
    'EGP': 1,
    'USD': 48.54,
    'EUR': 56.49,
    'SAR': 12.93,
    'AED': 13.22,
}

def parse_salary_record(val):
    text = str(val).lower().strip()

    number_match = re.search(r'\d+\.?\d*', text)
    if not number_match:
        return np.nan, None, np.nan

    number = float(number_match.group())

    if re.search(r'\d+\.?\d*\s*k', text) or re.search(r'\d+\.?\d*k', text):
        number *= 1000

    elif number < 160:
        number *= 1000

    currency_match = re.search(r'(usd|egp|sar|aed|eur|euro|dollar|dirham|\$|€)', text)

    if currency_match:
        currency = currency_match.group().upper()
        currency = {
            'DOLLAR': 'USD',
            '$': 'USD',
            'USD': 'USD',
            'EURO': 'EUR',
            '€': 'EUR',
            'EUR': 'EUR',
            'DIRHAM': 'AED',
            'AED': 'AED',
            'SAR': 'SAR',
            'EGP': 'EGP',
        }.get(currency, currency)
    else:
        currency = 'EGP'

    rate = conversion_rates_to_egp.get(currency, 1)
    amount_egp = number * rate

    return number, currency, amount_egp

df[['Amount', 'Currency', 'Amount_EGP']] = df['Salary'].apply(lambda x: pd.Series(parse_salary_record(x)))

In [1127]:
df.sort_values(by='Amount_EGP').head(60)

,Title,Years of Experiences,Salary,Date of Salary,What Is your Company,Work Type,Work Hour,City of Company site,Amount,Currency,Amount_EGP
756,Test,0,0,2025-02-01 00:00:00,Egyption,On Site,Full Time,Cai,0.0,EGP,0.0
380,Fullstack Developer (.Net/Angular),3,1000,2023-04-01 00:00:00,Not Egyption but site in egypt,Hybird,Full Time,Cairo,1000.0,EGP,1000.0
125,Cluster Revenue Analyst,3 years,1,2025-07-17 00:00:00,Egyption,On Site,Full Time,Cairo,1000.0,EGP,1000.0
607,Front End Vue/Nuxt Js,5,1100,2025-07-16 00:00:00,Not Egyption and site out of egypt,Remotley,Full Time,Jordan,1100.0,EGP,1100.0
731,Sr. Ui/Ux Designer | Product Designer,5,1100,2025-07-22 00:00:00,Egyption,Remotley,Full Time,Cairo,1100.0,EGP,1100.0
46,Ai Engineer,4,1150,2025-01-01 00:00:00,Not Egyption but site in egypt,On Site,Full Time,Giza,1150.0,EGP,1150.0
310,Frontend Developer,5,1550,2025-07-29 00:00:00,Not Egyption but site in egypt,Remotley,Full Time,cairo,1550.0,EGP,1550.0
491,Odoo Developer,5,1600,2025-01-01 00:00:00,Not Egyption and site out of egypt,Remotley,Full Time,Saudi Arabia,1600.0,EGP,1600.0
465,Ml Intern,0,3000,2025-08-01 00:00:00,Egyption,Hybird,Full Time,Cairo,3000.0,EGP,3000.0
79,Backend Developer,1,3000,2025-07-01 00:00:00,Egyption,Remotley,Part Time,"Maadi, Cairo",3000.0,EGP,3000.0


In [1145]:
df = df[df['Amount_EGP']>= 500]

In [1129]:
titles = df['Title'].unique()
titles

array(['. Net Developer', '.Net', '.Net Backend Developer',
       '.Net Developer', '.Net Developsr', '.Net Deveoper', '.Net Junior',
       'Accountant', 'Admin Officer', 'Advanced Services Engineer',
       'Ai & Data Science Engineer', 'Ai Developer', 'Ai Engineer',
       'Ai Rnd Engineer', 'Ams Layout Design Engineer', 'Android',
       'Android Developer', 'Android Developer Trainee',
       'Angular Developer', 'Application Support Specialist',
       'Associate Android Software Engineer',
       'Associate Cybersecurity Consultant',
       'Automotive Embedded Software Test Engineer', 'Back End',
       'Back End Developer', 'Back End Engineer Php Laravel',
       'Back End Php Developer', 'Backend', 'Backend ( Php )',
       'Backend And Devops Engineer (Nodejs, Java, Aws, Terraform)',
       'Backend Developer', 'Backend Developer (Django+Nestjs)',
       'Backend Developer - Php Laravel', 'Backend Developer Php Laravel',
       'Backend Developer Php/Laravel', 'Backend Engi

In [1131]:
categories = {
    "Embedded": ["embedded"],
    "Security": ["security", "cybersecurity"],
    "Fullstack": ["fullstack","WebDeveloper"],
    "UI UX": ["ui/ux", "ux\\ui", "ux",],
    "Mobile": ["mobile","Flutter","android", "ios", "application"],
    "Backend": ["backend", ".net", "dotnet", "php", "laravel", "nodejs", "django", "nestjs", "rubyonrails"],
    "Frontend": ["frontend", "angular", "react", "vue", "next","wordpress"],
    "Analyst": ["bi", "business", "revenue","Data Production Analyst"],
    "DevOps-MLOps": ["devops", "cloud"],
    "Biomedical": ["biomedical", "clinical"],
    "IT": ["it", "technical", "telecom", "broadcast", "customerservice", "communication"],
    "Engineering": ["civil", "estimation", "electronic", "controlengineer", "structuralsteeldetailer"],
    "Data-Science": ["data", "ai", "ml", "computervision", "scientist", "visualisation", "bigdata", "nlp", "sql"],
    "Testing": ["qaengineer", "test", "automation", "testing", "softwaretester", "sqc", "solution"]
}

THRESHOLD = 85

def assign_category(title:str):
    title_lower = title.lower().replace(" ","")
    matched_categories = []

    for category, keywords in categories.items():
        for kw in keywords:
            if fuzz.partial_ratio(title_lower, kw) >= THRESHOLD:
                matched_categories.append(category)
                break  

    return matched_categories[0] if matched_categories else "Other"

df['Category'] = df['Title'].apply(assign_category)


In [1147]:
df.reset_index()

,index,Title,Years of Experiences,Salary,Date of Salary,What Is your Company,Work Type,Work Hour,City of Company site,Amount,Currency,Amount_EGP,category
0,0,. Net Developer,6,1200 usd,2025-07-16 00:00:00,Not Egyption but site in egypt,Remotley,Full Time,Cairo,1200.0,USD,58248.0,Backend
1,1,. Net Developer,2,400 USD,2025-07-07 00:00:00,Not Egyption and site out of egypt,Remotley,Full Time,Riyadh,400.0,USD,19416.0,Backend
2,2,. Net Developer,6 months,13000,2025-07-18 00:00:00,Egyption,On Site,Full Time,Cairo,13000.0,EGP,13000.0,Backend
3,3,.Net,8month,10,2025-07-17 00:00:00,Egyption,Remotley,Full Time,Tanta,10000.0,EGP,10000.0,Backend
4,4,.Net Backend Developer,7 months,15000,2025-06-01 00:00:00,Egyption,On Site,Full Time,Giza,15000.0,EGP,15000.0,Backend
...,...,...,...,...,...,...,...,...,...,...,...,...,...
764,767,Warehouse Manager,7,25k net,2025-01-02 00:00:00,Egyption,On Site,Full Time,Giza,25000.0,EGP,25000.0,Other
765,768,Web Developer,0,8500,2025-01-01 00:00:00,Egyption,Remotley,Full Time,NaN,8500.0,EGP,8500.0,Other
766,769,Web Scraping Specialist,5,1500 $,2023-03-15 00:00:00,Not Egyption and site out of egypt,Hybird,Full Time,Kuwait,1500.0,USD,72810.0,Other
767,770,Wordpress Developer,2,10000,2024-10-01 00:00:00,Egyption,Remotley,Full Time,Cairo,10000.0,EGP,10000.0,Frontend


In [1187]:
df

,Title,Years of Experiences,Salary,Date of Salary,What Is your Company,Work Type,Work Hour,City of Company site,Amount,Currency,Amount_EGP,category
0,. Net Developer,6,1200 usd,2025-07-16 00:00:00,Not Egyption but site in egypt,Remotley,Full Time,Cairo,1200.0,USD,58248.0,Backend
1,. Net Developer,2,400 USD,2025-07-07 00:00:00,Not Egyption and site out of egypt,Remotley,Full Time,Riyadh,400.0,USD,19416.0,Backend
2,. Net Developer,6 months,13000,2025-07-18 00:00:00,Egyption,On Site,Full Time,Cairo,13000.0,EGP,13000.0,Backend
3,.Net,8month,10,2025-07-17 00:00:00,Egyption,Remotley,Full Time,Tanta,10000.0,EGP,10000.0,Backend
4,.Net Backend Developer,7 months,15000,2025-06-01 00:00:00,Egyption,On Site,Full Time,Giza,15000.0,EGP,15000.0,Backend
...,...,...,...,...,...,...,...,...,...,...,...,...
767,Warehouse Manager,7,25k net,2025-01-02 00:00:00,Egyption,On Site,Full Time,Giza,25000.0,EGP,25000.0,Other
768,Web Developer,0,8500,2025-01-01 00:00:00,Egyption,Remotley,Full Time,NaN,8500.0,EGP,8500.0,Other
769,Web Scraping Specialist,5,1500 $,2023-03-15 00:00:00,Not Egyption and site out of egypt,Hybird,Full Time,Kuwait,1500.0,USD,72810.0,Other
770,Wordpress Developer,2,10000,2024-10-01 00:00:00,Egyption,Remotley,Full Time,Cairo,10000.0,EGP,10000.0,Frontend


In [1205]:
df = df.rename(columns={"Years of Experiences ": "Experience"})

In [1213]:
def round_to_half(x):
    return round(x * 2) / 2

def parse_experience(val):
    if pd.isna(val):
        return np.nan
    text = str(val).strip().lower()

    text = re.sub(r'\(.*?\)', '', text).strip()

    try:
        numeric_val = float(re.sub(r'[^0-9\.-]', '', text) or 0)
    except ValueError:
        numeric_val = 0

    if 'fresh' in text or 'junior' in text or numeric_val < 0:
        return np.nan

    if any(m in text for m in ['month', 'mo']):
        num = re.findall(r'[\d\.]+', text)
        if num:
            try:
                months = float(num[0])
                return round_to_half(months / 12)
            except ValueError:
                return np.nan
        return np.nan

    if 'less than' in text or '<' in text:
        num = re.findall(r'[\d\.]+', text)
        if num:
            try:
                return round_to_half(max(float(num[0]) - 0.1, 0))
            except ValueError:
                return np.nan
        return np.nan

    if 'half' in text:
        return 0.5

    if 'almost' in text:
        num = re.findall(r'[\d\.]+', text)
        if num:
            try:
                return round_to_half(float(num[0]) - 0.1)
            except ValueError:
                return np.nan
        return np.nan

    num = re.findall(r'[\d\.]+', text)
    if num:
        try:
            return round_to_half(float(num[0]))
        except ValueError:
            return np.nan

    return np.nan


df["Experience Years"] = df["Experience"].apply(parse_experience)

In [1215]:
df

,Title,Experience,Salary,Date of Salary,What Is your Company,Work Type,Work Hour,City of Company site,Amount,Currency,Amount_EGP,category,Experience Years
0,. Net Developer,6,1200 usd,2025-07-16 00:00:00,Not Egyption but site in egypt,Remotley,Full Time,Cairo,1200.0,USD,58248.0,Backend,6.0
1,. Net Developer,2,400 USD,2025-07-07 00:00:00,Not Egyption and site out of egypt,Remotley,Full Time,Riyadh,400.0,USD,19416.0,Backend,2.0
2,. Net Developer,6 months,13000,2025-07-18 00:00:00,Egyption,On Site,Full Time,Cairo,13000.0,EGP,13000.0,Backend,0.5
3,.Net,8month,10,2025-07-17 00:00:00,Egyption,Remotley,Full Time,Tanta,10000.0,EGP,10000.0,Backend,0.5
4,.Net Backend Developer,7 months,15000,2025-06-01 00:00:00,Egyption,On Site,Full Time,Giza,15000.0,EGP,15000.0,Backend,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
767,Warehouse Manager,7,25k net,2025-01-02 00:00:00,Egyption,On Site,Full Time,Giza,25000.0,EGP,25000.0,Other,7.0
768,Web Developer,0,8500,2025-01-01 00:00:00,Egyption,Remotley,Full Time,NaN,8500.0,EGP,8500.0,Other,0.0
769,Web Scraping Specialist,5,1500 $,2023-03-15 00:00:00,Not Egyption and site out of egypt,Hybird,Full Time,Kuwait,1500.0,USD,72810.0,Other,5.0
770,Wordpress Developer,2,10000,2024-10-01 00:00:00,Egyption,Remotley,Full Time,Cairo,10000.0,EGP,10000.0,Frontend,2.0


In [1219]:
df.to_csv(r"C:\Users\mfgas\OneDrive\Desktop\Salary.csv")